In [1]:
import pandas as pd
from xml.dom import minidom
document = minidom.parse("dataset.xml")

In [2]:
# Find the pattern in the sequence
def getPattern(sequence, position):
    start_str_index = position - 20

    if start_str_index < 0:
        start_str_index = 0

    start_str = sequence[start_str_index:position]
    middle_str = sequence[position]
    after_str = sequence[position+1:position+1+20]

    if len(start_str) < 20:
        x_count = 20 - len(start_str)
        start_str = 'X' * x_count + start_str

    if len(after_str) < 20:
        x_count = 20 - len(after_str)
        after_str = after_str + 'X' * x_count

    return start_str + middle_str + after_str

In [3]:
# The type and description in your search criteria
search_type = "modified residue"
#search_descriptions = ["N-acetylthreonine"]

# Result that will be created once a match is found
sequences = []
types = []
positions = []
#descriptions = []
patterns = []

entries = document.getElementsByTagName("entry")
for i, entry in enumerate(entries):
    features = entry.getElementsByTagName("feature")
    sequence = entry.getElementsByTagName("sequence")

    length = sequence[0].attributes.get("length", None)
    if length != None:
        # Get sequence body
        data = sequence[0].childNodes[0].data

        # Get positions for the sequence.
        for i, feature in enumerate(features):
            if feature.attributes["type"].value == search_type: 
            #and feature.attributes["description"].value in search_descriptions:
                position = feature.getElementsByTagName("position")
                position = int(position[0].attributes["position"].value)
                pattern = getPattern(data, position-1)
                
                # Append the data
                sequences.append(data)
                types.append(feature.attributes["type"].value)
                positions.append(position)
                #descriptions.append(feature.attributes["description"].value)
                patterns.append(pattern)

In [4]:
# Create a DataFrame and save the data to the disk.
df = pd.DataFrame({
    'sequence': pd.Series(sequences),
    'type': pd.Series(types),
    'position': pd.Series(positions),
    #'description': pd.Series(descriptions),
    'pattern': pd.Series(patterns)
})
df.to_csv("normalization-data.csv", index=False)